# Fuel-Efficient Route Model Training and Saving

This notebook does the following:
1. Loads and preprocesses your vehicle and traffic data.
2. Trains two models:
   - Fuel Consumption Model
   - CO₂ Emissions Model
3. Saves the trained models as `.h5` files in a folder called `models/`.

Run all the cells to generate the models. Once saved, these files can be used by your Flask API.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# --- 1. Load Your Data ---
vehicle_data = pd.read_csv('data/vehicle data/vehicles.csv')
traffic_volumes = pd.read_csv('data/traffic data/Traffic_Volumes.csv')
bottlenecks = pd.read_csv('data/traffic data/Bottlenecks.csv')

print("Vehicle Data Sample:")
print(vehicle_data.head())
print("\nTraffic Volumes Sample:")
print(traffic_volumes.head())
print("\nBottlenecks Sample:")
print(bottlenecks.head())

# --- 2. Data Cleaning and Preprocessing ---
vehicle_data.dropna(inplace=True)
traffic_volumes.dropna(inplace=True)
bottlenecks.dropna(inplace=True)

vehicle_data.rename(columns={
    'Fuel_Consumption_City': 'city_fuel_efficiency',
    'Fuel_Consumption_Hwy': 'highway_fuel_efficiency',
    'CO2_Emissions': 'co2_emissions'
}, inplace=True)

vehicle_data['combined_fuel_efficiency'] = (
    vehicle_data['city_fuel_efficiency'] * 0.55 +
    vehicle_data['highway_fuel_efficiency'] * 0.45
)

bottlenecks['normalized_traffic_severity'] = (
    bottlenecks['traffic_severity'] / bottlenecks['traffic_severity'].max()
)

# --- 3. Merge Datasets ---
traffic_data = pd.merge(
    traffic_volumes,
    bottlenecks[['location', 'normalized_traffic_severity']],
    on='location',
    how='left'
)

merged_data = pd.merge(
    traffic_data,
    vehicle_data,
    on='Fuel_Type',
    how='left'
)

merged_data['fuel_consumption'] = merged_data['distance'] / merged_data['combined_fuel_efficiency']

# --- 4. Prepare Features and Targets ---
X = merged_data[['distance', 'normalized_traffic_severity', 'combined_fuel_efficiency']]
y_fuel = merged_data['fuel_consumption']   # Target for fuel consumption
y_co2 = merged_data['co2_emissions']         # Target for CO₂ emissions

# --- 5. Train-Test Split and Scaling ---
X_train, X_test, y_fuel_train, y_fuel_test = train_test_split(X, y_fuel, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X, y_co2, test_size=0.2, random_state=42)

scaler_fuel = StandardScaler()
X_train_fuel = scaler_fuel.fit_transform(X_train)
X_test_fuel = scaler_fuel.transform(X_test)

scaler_co2 = StandardScaler()
X_train_co2 = scaler_co2.fit_transform(X_train)
X_test_co2 = scaler_co2.transform(X_test)

# --- 6. Define a Model Builder Function ---
def build_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mse'])
    return model

# --- 7. Train the Models ---
print("Training Fuel Consumption Model...")
model_fuel = build_model(X_train_fuel.shape[1])
model_fuel.fit(X_train_fuel, y_fuel_train, validation_split=0.2, epochs=30, batch_size=32, verbose=1)

print("\nTraining CO₂ Emissions Model...")
model_co2 = build_model(X_train_co2.shape[1])
model_co2.fit(X_train_co2, y_co2_train, validation_split=0.2, epochs=30, batch_size=32, verbose=1)

# --- 8. Save the Trained Models ---
os.makedirs('models', exist_ok=True)
model_fuel.save('models/fuel_consumption_model.h5')
model_co2.save('models/co2_emissions_model.h5')

print("Models saved in the 'models/' folder.")
